### Integer Programming - Knapsack

Maximize the total profit by selecting items such that their total weight does not exceed the knapsack's capacity.

Problem:

$$\text{Maximize} \quad Z = \sum_{i=1}^{N} v_i x_i$$

Subject to:

$$\sum_{i=1}^{N} w_i x_i \leq W$$

Where $x_i \in \{0, 1\}$, $v_i$ is the value, $w_i$ is the weight, and $W$ is the weight capacity.

In [1]:
# pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import numpy as np
N = 4
v = np.array([15, 10, 25, 30])
w = np.array([4, 3, 5, 6])
W = 10
model = LpProblem("Knapsack", LpMaximize)
x = [LpVariable(f"x{i}", cat="Binary") for i in range(N)]
model += lpSum(v[i] * x[i] for i in range(N)), "Total_Profit"
model += lpSum(w[i] * x[i] for i in range(N)) <= W, "Size_Capacity"
status = model.solve()
x_values = [x[i].value() for i in range(N)]  # Extract values of decision variables
chosen = np.where(np.array(x_values) == 1)[0]  # Indices of selected items
max_profit = v[chosen].sum()
used_space = w[chosen].sum()
print(x_values, max_profit, used_space)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /econ_share/home/pp712/myenv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/457900edb789438b8086bd94891345e4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/457900edb789438b8086bd94891345e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 23 RHS
At line 25 BOUNDS
At line 30 ENDATA
Problem MODEL has 1 rows, 4 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 50 - 0.00 seconds
Cgl0004I processed model has 1 rows, 4 columns (4 integer (4 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.2
Cbc0038I Pass   1: suminf.    0.16667 (1) obj. -50 iterations 1
Cbc0038I Solution found of -25
Cbc0038I Rounding solution of -40 is better than pre

### Mixed Integer Programming

Maximize x + 10y subject to the following constraints:
- x + 7y ≤ 17.5
- 0 ≤ x ≤ 3.5
- 0 ≤ y
- x, y integers

In [2]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver("SAT") # CP-SAT
infinity = solver.infinity()
x = solver.IntVar(0.0, infinity, "x")
y = solver.IntVar(0.0, infinity, "x")
solver.Add(x + 7 * y <= 17.5)
solver.Add(x <= 3.5)
solver.Maximize(x + 10 * y)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(solver.Objective().Value())
    print(x.solution_value())
    print(y.solution_value())
    #print(solver.wall_time()) # milliseconds
    #print(solver.iterations())
    #print(solver.nodes())
    #print(solver.NumConstraints())

23.0
3.0
2.0


In [4]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver("GLOP") # CP-SAT
infinity = solver.infinity()
x = solver.NumVar(0.0, infinity, "x")
y = solver.NumVar(0.0, infinity, "x")
solver.Add(x + 7 * y <= 17.5)
solver.Add(x <= 3.5)
solver.Maximize(x + 10 * y)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(solver.Objective().Value())
    print(x.solution_value())
    print(y.solution_value())
    #print(solver.wall_time()) # milliseconds
    #print(solver.iterations())
    #print(solver.nodes())
    #print(solver.NumConstraints())

25.0
0.0
2.5


Maximize 7x1 + 8x2 + 2x3 + 9x4 + 6x5 subject to the following constraints:

- 5 x1 + 7 x2 + 9 x3 + 2 x4 + 1 x5 ≤ 250
- 18 x1 + 4 x2 - 9 x3 + 10 x4 + 12 x5 ≤ 285
- 4 x1 + 7 x2 + 3 x3 + 8 x4 + 5 x5 ≤ 211
- 5 x1 + 13 x2 + 16 x3 + 3 x4 - 7 x5 ≤ 315

where x1, x2, ..., x5 are non-negative integers.



In [7]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver("SAT")

def create_data_model():
    data = {}
    data["constraint_coeffs"] = [
        [5, 7, 9, 2, 1], 
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -5]
    ]
    data["bounds"] = [250, 285, 211, 315]
    data["obj_coeffs"] = [7, 8, 2, 9, 6]
    data["num_vars"] = 5
    data["num_constraints"] = 4
    return data

data = create_data_model()
solver = pywraplp.Solver.CreateSolver("SCIP")
infinity = solver.infinity()
x = {}
for j in range(data["num_vars"]):
    x[j] = solver.IntVar(0, infinity, "x[%i]" % j)
print("Number of variables = ", solver.NumVariables())

for i in range(data["num_constraints"]):
    constraint = solver.RowConstraint(0, data["bounds"][i], "")
    for j in range(data["num_vars"]):
        constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j])
print("Number of constraints", solver.NumConstraints())

objective = solver.Objective()
for j in range(data["num_vars"]):
    objective.SetCoefficient(x[j], data["obj_coeffs"][j])
objective.SetMaximization()

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print(solver.Objective().Value())
    for j in range(data["num_vars"]):
        print(x[j].name(), " = ", x[j].solution_value())
    print(solver.wall_time()) # milliseconds
    print(solver.iterations())
    print(solver.nodes())

Number of variables =  5
Number of constraints 4
260.00000000000006
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0
13
42
1
